In [2]:
import pandas as pd
from sodapy import Socrata
import warnings
warnings.simplefilter('ignore')
# Display Setting
from IPython.display import display
pd.options.display.max_colwidth=100
pd.options.display.float_format="{:.2f}".format
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as pl
import chart_studio.plotly as py
import cufflinks as cf
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
from sklearn.metrics import auc,roc_curve,classification_report,confusion_matrix,mean_absolute_error,mean_squared_error,root_mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import nltk
import string
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
%matplotlib inline

In [ ]:
""" Set up a for loop to iterate over each feature
Print the name of the feature (excel header)
Plot how many are empty/null/na/0
Print data type
If categorical:
Print top values, print unique values [na, 'missing', 'did not provide']
If numerical:
Print max, mean, plot distributions [ages may be off, duration may not make sense]
Put together a rough data dictionary, provide to stakeholders for feedback (share the cleanup load!)
Put together a correlation matrix
Spearman
Pearson
"""

#### Read data

In [7]:
animal_intakes_raw_data = pd.read_csv("Austin_Animal_Center_Intakes.csv")
animal_outcome_raw_data = pd.read_csv("Austin_Animal_Center_Outcomes.csv")

In [6]:
animal_intakes_raw_data

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
0,A521520,Nina,10/01/2013 07:51:00 AM,October 2013,Norht Ec in Austin (TX),Stray,Normal,Dog,Spayed Female,7 years,Border Terrier/Border Collie,White/Tan
1,A664235,NaN,10/01/2013 08:33:00 AM,October 2013,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White
2,A664236,NaN,10/01/2013 08:33:00 AM,October 2013,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White
3,A664237,NaN,10/01/2013 08:33:00 AM,October 2013,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White
4,A664233,Stevie,10/01/2013 08:53:00 AM,October 2013,7405 Springtime in Austin (TX),Stray,Injured,Dog,Intact Female,3 years,Pit Bull Mix,Blue/White
...,...,...,...,...,...,...,...,...,...,...,...,...
173807,A929690,NaN,05/03/2025 11:18:00 PM,May 2025,8038 Exchange Dr in Austin (TX),Stray,Injured,Dog,Intact Male,2 years,Belgian Malinois,Brown/Black
173808,A929717,NaN,05/04/2025 03:14:00 PM,May 2025,Austin (TX),Public Assist,Normal,Dog,Intact Male,1 year,Shih Tzu Mix,White/Blue
173809,A929724,NaN,05/04/2025 07:43:00 PM,May 2025,7105 Providence Ave Apt 3 in Austin (TX),Stray,Normal,Other,Unknown,1 year,Rabbit Sh,Tan/White
173810,A929725,Oswold,05/04/2025 10:55:00 PM,May 2025,1501 Red River St in Austin (TX),Public Assist,Normal,Dog,Intact Male,10 years,Boxer Mix,Tan/White


In [8]:
animal_outcome_raw_data

,Animal ID,Date of Birth,Name,DateTime,MonthYear,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A668305,2012-12-01,NaN,2013-12-02T00:00:00-05:00,12-2013,Transfer,Partner,Other,Unknown,1 year,Turtle Mix,Brown/Yellow
1,A673335,2012-02-22,NaN,2014-02-22T00:00:00-05:00,02-2014,Euthanasia,Suffering,Other,Unknown,2 years,Raccoon,Black/Gray
2,A675999,2013-04-03,NaN,2014-04-07T00:00:00-05:00,04-2014,Transfer,Partner,Other,Unknown,1 year,Turtle Mix,Green
3,A679066,2014-04-16,NaN,2014-05-16T00:00:00-05:00,05-2014,NaN,NaN,Other,Unknown,4 weeks,Rabbit Sh,Brown
4,A680855,2014-05-25,NaN,2014-06-10T00:00:00-05:00,06-2014,Transfer,Partner,Bird,Unknown,2 weeks,Duck,Yellow/Black
...,...,...,...,...,...,...,...,...,...,...,...,...
173770,A671565,2013-07-06,NaN,2014-02-07T10:14:00,02-2014,Transfer,SCRP,Cat,Spayed Female,7 months,Domestic Medium Hair Mix,Brown Tabby
173771,A675119,2014-02-16,NaN,2014-03-27T08:00:00,03-2014,Transfer,Partner,Dog,Intact Male,1 month,Pomeranian Mix,White/Brown
173772,A677839,2014-01-29,NaN,2014-05-03T08:00:00,05-2014,Transfer,Partner,Other,Unknown,3 months,Lop-English Mix,Chocolate
173773,A678559,2011-05-10,Gatsby,2014-05-14T08:00:00,05-2014,Transfer,Partner,Dog,Intact Male,3 years,Pekingese/Lhasa Apso,Buff
